In [1]:
# Import necessary packages
from pydantic import BaseModel, ValidationError, Field, EmailStr
from typing import List, Literal, Optional
import json
from datetime import date
from dotenv import load_dotenv
import openai

In [2]:
# Load environment variables for API access
load_dotenv()
# Initialize OpenAI client for API calls
client = openai.OpenAI()

In [17]:
# Define a function to call the LLM
def call_llm(prompt, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [18]:
pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import pdfplumber

def extract_pdf_text_to_variable(pdf_path):
    text_data = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text_data += page_text + "\n"
    return text_data

# Usage example
pdf_file = "SLV053.pdf"   # Replace with your PDF file path
md_text = extract_pdf_text_to_variable(pdf_file)

print(md_text)  # Now pdf_text variable contains all the PDF text

An official website of the United States government Here's how you know
MENU
Export License Application
Status COMPLETED - APPROVED W/CONDITIONS
Contact Information
Reference Number
SLV0530
1. Contact Person (First Name, Last Name)
Shelley Vybiral
2. Telephone Number 3. Fax Number
6302003543 -
Email
shelley.vybiral@cmcelectronics.us
4. Creation Date
05/30/2025
5. Type of Application
Export License Application
Document Checklist
6. Documents submitted with application
✔ Export Items (BIS-748P-A)
End Users (BIS-748P-B)
BIS-711
✔ Import/End-User Certificate
Technical Specification
Letter of Explanation
Foreign Availability
Other Tell us what to improve
purchase order
7. Documents on file with applicant
BIS-711
Letter of Assurance
Import/End-User Certificate
Nuclear Certification
Other
-
License Information
9. Special Purpose
-
10. Resubmission ACN 11. Replacement License Number
- -
13. Import Certificate Country Import Certificate Number
- -
Applicant Information
14. CIN (Applicant ID) Ap

In [20]:
prompt = f"""
You are a document parser. Your task is to convert the provided PDF text and form data
into structured JSON.

Rules:

1. Include all fields in the output JSON, even if their value is empty or null.
Understand the file structure.
2. Input may contain checkbox fields, understand how it is represented.
For checkboxes or radio buttons:
   -Include only the items that are selected (marked with '✔' or similar).
   - If no items in a checkbox/radio group are selected, set the field to null.
   - Do NOT output boolean true/false for unselected items.
   - Extract only the items that are marked as selected, indicated by a leading '✔' or similar mark.
    - Ignore items without any selection mark.
    - If no items are selected in a checklist, set it to null.
    - Include only the items that are marked as selected, indicated by a leading '✔', or 'n' or similar mark.
  - Do NOT output boolean true/false flags for items.
  - If no items are selected in a checklist, set that checklist field to null.
  - if checkbox is under different column in table, keep it under that column name. You have follow the structure.

3. Split full names into 'FirstName' and 'LastName' if possible.
4. Group logically related fields together.
5. Output valid JSON only. No explanation, no extra text.
6.Extract data if it is split into two coloumns and/or numbered columns data

Here are the filled form fields (checkboxes, radios, etc.):

{md_text}

Here is the extracted text from the PDF:

{md_text}
"""
prompt_ = prompt.format(md_text=md_text)

In [21]:
# Get response from LLM
# print(prompt_)
import re
from pprint import pprint
response_content = call_llm(prompt_)
pattern = r'<text>(.*?)</text>'
match = re.search(pattern, response_content, re.DOTALL)
if match:
    resp = json.loads(match.group(1))
    pprint(resp)
    print(resp.keys())
else:
    print(response_content)

```json
{
  "ExportLicenseApplication": {
    "Status": "COMPLETED - APPROVED W/CONDITIONS",
    "ContactInformation": {
      "ReferenceNumber": "SLV0530",
      "ContactPerson": {
        "FirstName": "Shelley",
        "LastName": "Vybiral"
      },
      "TelephoneNumber": "6302003543",
      "FaxNumber": null,
      "Email": "shelley.vybiral@cmcelectronics.us",
      "CreationDate": "05/30/2025",
      "TypeOfApplication": "Export License Application"
    },
    "DocumentChecklist": {
      "DocumentsSubmittedWithApplication": [
        "Export Items (BIS-748P-A)",
        "Import/End-User Certificate",
        "purchase order"
      ],
      "DocumentsOnFileWithApplicant": null
    },
    "LicenseInformation": {
      "SpecialPurpose": null,
      "ResubmissionACN": null,
      "ReplacementLicenseNumber": null,
      "ImportCertificate": {
        "Country": null,
        "ImportCertificateNumber": null
      }
    },
    "ApplicantInformation": {
      "CIN": "C702375",
      "A